In [12]:
#!import AutomatedAnalysis.dib

  Determining projects to restore...
  All projects are up-to-date for restore.
  GC.Analysis.API -> C:\Workspace\dev\performance-dev\artifacts\bin\GC.Analysis.API\Release\net8.0\GC.Analysis.API.dll

Build succeeded.
    0 Warning(s)
    0 Error(s)

Time Elapsed 00:00:01.40


Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-public/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.19.1 Microsoft.Diagnostics.Tracing.TraceEvent, 3.1.13 XPlot.Plotly, 4.0.6 XPlot.Plotly.Interactive, 4.0.7

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-public/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.19.1 Microsoft.Diagnostics.Tracing.TraceEvent, 3.1.13 XPlot.Plotly, 4.0.6 XPlot.Plotly.Interactive, 4.0.7

In [25]:
string path = @".\Examples\Traces\All_Conserve";
DataManager dm = DataManager.CreateAspNetData(path);
string baselineConfigName = "28c";
string comparandConfigName = "28c_con1";

// Set to null if summary all benchmarks
Filter benchmarkFilter = new Filter("Stage1Pgo"); 

## Display Benchmark Regressions

In [14]:
Metric<BenchmarkData> avgHeapCountChanges = Metrics.Promote(Metrics.I.NumberOfHeapCountSwitches, Aggregation.Average);
Metric<BenchmarkData> avgGCCount = Metrics.Promote(Metrics.I.TotalGCCount, Aggregation.Average);
public static Dictionary<Metric<BenchmarkData>, ExpectationDirection> HigherMemoryLowerThroughput = new()
{
    { Metrics.B.AverageMaxHeapSize, ExpectationDirection.Increase },
    { Metrics.B.AverageP50Latency, ExpectationDirection.Decrease },
    /*
    Additionally, a metric with no change is also available. The entire enum is:
    public enum ExpectationDirection
    {
        Unknown,
        Increase,
        Decrease,
        NoChange,
    }
    */
};

Dictionary< Metric<BenchmarkData>, double> thresholds = new()
{
    { Metrics.B.AverageMaxHeapSize, 5 },
    { Metrics.B.AverageP50Latency, 5 },
};

DisplayBenchmarkSummaryWithRegressions(dm, baselineConfigName, comparandConfigName, metrics: ML( Metrics.B.AverageMaxHeapSize, Metrics.B.AverageP50Latency ), benchmarkRegressionThreshold: 5, benchmarksToDisplay: 5, 
metricToleranceOverridePercentage: thresholds,
expectationDirections: HigherMemoryLowerThroughput);

## Out of 30 benchmarks, 6 have regressed.
### Thresholds Used
- Default Threshold: 5%
- Average of Max heap size: 5%
- Average of Latency 50th: 5%
### Details
   - __ConnectionClose__ has regressed by 24.77% for Average of Latency 50th
   - __JsonHttpSys__ has regressed by 17.54% for Average of Latency 50th
   - __Json__ has regressed by 12.09% for Average of Latency 50th
   - __Stage1Pgo__ has regressed by 5.31% for Average of Latency 50th
   - __ConnectionClose__ has regressed by -5.24% for Average of Max heap size


### Per-benchmark behavior

|          Benchmark Name | Average of Max heap size / 28c | Average of Max heap size / 28c_con1 | Average of Max heap size /  Comparison % | Average of Latency 50th / 28c | Average of Latency 50th / 28c_con1 | Average of Latency 50th /  Comparison % |
| ----------------------: | -----------------------------: | ----------------------------------: | ---------------------------------------: | ----------------------------: | ---------------------------------: | --------------------------------------: |
|         ConnectionClose |                          7.575 |                               7.178 |                                   -5.241 |                         1.100 |                              1.373 |                                  24.773 |
|             JsonHttpSys |                         34.516 |                              48.246 |                                   39.780 |                         0.527 |                              0.620 |                                  17.536 |
|                    Json |                          4.110 |                               4.102 |                                   -0.186 |                         0.228 |                              0.255 |                                  12.088 |
| PlaintextPlatformInline |                          7.521 |                               5.877 |                                  -21.858 |                         1.038 |                              1.045 |                                   0.723 |
|       PlaintextPlatform |                          5.158 |                               3.234 |                                  -37.314 |                         0.862 |                              0.853 |                                  -1.159 |
|               Stage1Pgo |                         30.603 |                              38.501 |                                   25.807 |                         0.283 |                              0.298 |                                   5.310 |

## Display Iteration Anomalies

In [15]:
// Update Average -> Safe Average.
Aggregation SafeAverage = new Aggregation((gc => {
    double safeAverage;
    
    long count = 0;
    double sum = 0;

    foreach (var item in gc)
    {
        if (item != 0 && !double.IsNaN(item))
        {
            sum += item;
            count++;
        }
    }

    safeAverage = count == 0 ? double.NaN : sum / count;
    return safeAverage;

}), "SafeAverage", "");

Metric<IterationData> avgTCPToConsider = Metrics.Promote(Metrics.G.MedianThroughputCostPercent, SafeAverage);

Dictionary<Metric<IterationData>, double> volatilityOverrides = new()
{
    { Metrics.I.Gen0MeanPauseMSec, 10 },
    { Metrics.I.Latency50thMS, 20 },
};

// todo: override the volatility threshold
DisplayIterationData(dm, baselineConfigName, comparandConfigName, metrics: ML( avgTCPToConsider, Metrics.I.Gen0MeanPauseMSec, Metrics.I.Latency50thMS ) //, benchmarkFilter: new Filter("Stage1Pgo"),
,volatilityThreshold: 2, metricVolOverridePercentage: volatilityOverrides, iterationsToDisplay: 5) 

## Anomalies in Iterations
### Thresholds
- Default Volatility Threshold: 2%
- g0 Mean Pause: 10%
- Latency 50th: 20%
### Details
   - __28c-ConnectionClose__ for Latency 50th has an anomaly 0.31 for iteration __3__ with Volatility: 393.55%:  1.24, 1.53, 1.32, 0.31.
   - __28c_con1-PlaintextPlatformInline__ for g0 Mean Pause has an anomaly 19.52 for iteration __1__ with Volatility: 256.73%:  5.57, 19.52, 6.08, 5.47.
   - __28c-JsonHttpSys__ for Latency 50th has an anomaly 0.75 for iteration __3__ with Volatility: 74.42%:  0.43, 0.5, 0.43, 0.75.
   - __28c-PlaintextWithParameterstWithFilter__ for SafeAverage of MedianThroughputCostPercent has an anomaly 7.26 for iteration __1__ with Volatility: 49.54%:  5.39, 7.26, 4.86, 5.65.
   - __28c_con1-Stage1Pgo__ for SafeAverage of MedianThroughputCostPercent has an anomaly 1.4 for iteration __0__ with Volatility: 45.68%:  1.4, 2.04, 1.86, 1.83.
### Bimodal Behavior in Iterations
   - __28c-PlaintextQueryString__ for SafeAverage of MedianThroughputCostPercent has bimodal behavior with __2__ bimodal clusters with Volatility: 103.85%:  3.63, 3.79, 7.2, 7.4 - (7.2, 7.4),(3.63, 3.79).
   - __28c_con1-PlaintextWithParameterstWithFilter__ for SafeAverage of MedianThroughputCostPercent has bimodal behavior with __2__ bimodal clusters with Volatility: 71.87%:  4.07, 4.08, 3.08, 2.37 - (4.07, 4.08),(3.08, 2.37).
   - __28c_con1-JsonHttpSys__ for Latency 50th has bimodal behavior with __2__ bimodal clusters with Volatility: 61.7%:  0.47, 0.55, 0.7, 0.76 - (0.7, 0.76),(0.47, 0.55).
   - __28c-ConnectionCloseHttps__ for SafeAverage of MedianThroughputCostPercent has bimodal behavior with __2__ bimodal clusters with Volatility: 56.1%:  1.55, 2.41, 1.96, 2.14 - (1.55, 1.96),(2.41, 2.14).
   - __28c_con1-JsonHttpSys__ for SafeAverage of MedianThroughputCostPercent has bimodal behavior with __2__ bimodal clusters with Volatility: 48.87%:  5.28, 5.02, 4.15, 3.55 - (5.28, 5.02),(4.15, 3.55).


## ConnectionCloseHttpsHttpSys

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | 1.98 | 1.89 | 0.26 |
| 28c | 1 | 1.98 | 1.90 | 0.26 |
| 28c | 2 | 2.20 | 1.71 | 0.27 |
| 28c | 3 | 2.22 | 1.72 | 0.28 |
| 28c_con1 | 0 | 1.93 | 1.90 | 0.26 |
| 28c_con1 | 1 | 1.95 | 1.93 | 0.27 |
| 28c_con1 | 2 | 2.24 | 1.74 | 0.27 |
| 28c_con1 | 3 | 2.20 | 1.72 | 0.27 |
| 28c | Average | 2.10 | 1.81 | 0.27 |
| 28c_con1 | Average | 2.08 | 1.82 | 0.27 |
| Comparison | % Diff | -0.71 | 0.89 | 0.00 |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c | 3 | Latency 50th  | 7.69 | 0.28 | 0.26 | 0.26 | 0.27 | 0.28 | 
| 28c_con1 | 0 | Latency 50th  | 3.85 | 0.26 | 0.26 | 0.27 | 0.27 | 0.27 | 
### Bimodal Behavior

| Config | Clusters | Metric | Vol % | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ------ | ------ | ------ | ------ |
| 28c_con1 | (2.24, 2.2),(1.93, 1.95) | SafeAverage of MedianThroughputCostPercent  | 16.12 | 1.93 | 1.95 | 2.24 | 2.20 | 
| 28c_con1 | (1.9, 1.93),(1.74, 1.72) | g0 Mean Pause  | 12.75 | 1.90 | 1.93 | 1.74 | 1.72 | 
| 28c | (2.2, 2.22),(1.98, 1.98) | SafeAverage of MedianThroughputCostPercent  | 12.34 | 1.98 | 1.98 | 2.20 | 2.22 | 
| 28c | (1.89, 1.9),(1.71, 1.72) | g0 Mean Pause  | 11.03 | 1.89 | 1.90 | 1.71 | 1.72 | 
## ConnectionCloseHttpSys

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | 2.32 | 1.50 | 0.19 |
| 28c | 1 | 2.31 | 1.48 | 0.19 |
| 28c | 2 | 1.80 | 1.32 | 0.19 |
| 28c | 3 | 1.85 | 1.32 | 0.19 |
| 28c_con1 | 0 | 2.31 | 1.51 | 0.19 |
| 28c_con1 | 1 | 2.35 | 1.51 | 0.19 |
| 28c_con1 | 2 | 2.37 | 1.50 | 0.19 |
| 28c_con1 | 3 | 2.37 | 1.53 | 0.19 |
| 28c | Average | 2.07 | 1.41 | 0.19 |
| 28c_con1 | Average | 2.35 | 1.51 | 0.19 |
| Comparison | % Diff | 13.70 | 7.75 | 0.00 |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c_con1 | 0 | SafeAverage of MedianThroughputCostPercent  | 2.86 | 2.31 | 2.31 | 2.35 | 2.37 | 2.37 | 
| 28c_con1 | 3 | g0 Mean Pause  | 1.85 | 1.53 | 1.51 | 1.51 | 1.50 | 1.53 | 
### Bimodal Behavior

| Config | Clusters | Metric | Vol % | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ------ | ------ | ------ | ------ |
| 28c | (1.8, 1.85),(2.32, 2.31) | SafeAverage of MedianThroughputCostPercent  | 29.1 | 2.32 | 2.31 | 1.80 | 1.85 | 
| 28c | (1.5, 1.48),(1.32, 1.32) | g0 Mean Pause  | 13.95 | 1.50 | 1.48 | 1.32 | 1.32 | 
## ConnectionCloseHttps

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | 1.55 | 1.21 | 0.19 |
| 28c | 1 | 2.41 | 1.74 | 0.18 |
| 28c | 2 | 1.96 | 1.44 | 0.18 |
| 28c | 3 | 2.14 | 1.44 | 0.18 |
| 28c_con1 | 0 | 2.15 | 1.50 | 0.19 |
| 28c_con1 | 1 | 2.57 | 1.75 | 0.19 |
| 28c_con1 | 2 | 2.01 | 1.43 | 0.19 |
| 28c_con1 | 3 | 1.77 | 1.21 | 0.18 |
| 28c | Average | 2.02 | 1.46 | 0.18 |
| 28c_con1 | Average | 2.13 | 1.48 | 0.19 |
| Comparison | % Diff | 5.41 | 1.21 | 2.74 |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c_con1 | 1 | g0 Mean Pause  | 44.46 | 1.75 | 1.50 | 1.75 | 1.43 | 1.21 | 
| 28c | 1 | g0 Mean Pause  | 43.23 | 1.74 | 1.21 | 1.74 | 1.44 | 1.44 | 
| 28c_con1 | 3 | Latency 50th  | 5.56 | 0.18 | 0.19 | 0.19 | 0.19 | 0.18 | 
### Bimodal Behavior

| Config | Clusters | Metric | Vol % | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ------ | ------ | ------ | ------ |
| 28c | (1.55, 1.96),(2.41, 2.14) | SafeAverage of MedianThroughputCostPercent  | 56.1 | 1.55 | 2.41 | 1.96 | 2.14 | 
| 28c_con1 | (2.15, 2.57),(2.01, 1.77) | SafeAverage of MedianThroughputCostPercent  | 45.05 | 2.15 | 2.57 | 2.01 | 1.77 | 
## ConnectionClose

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | 2.50 | 0.70 | 1.24 |
| 28c | 1 | 2.37 | 0.74 | 1.53 |
| 28c | 2 | 2.47 | 0.72 | 1.32 |
| 28c | 3 | 2.49 | 0.71 | 0.31 |
| 28c_con1 | 0 | 2.58 | 0.70 | 1.37 |
| 28c_con1 | 1 | 2.41 | 0.70 | 1.31 |
| 28c_con1 | 2 | 1.98 | 0.72 | 1.55 |
| 28c_con1 | 3 | 2.48 | 0.71 | 1.26 |
| 28c | Average | 2.46 | 0.72 | 1.10 |
| 28c_con1 | Average | 2.36 | 0.71 | 1.37 |
| Comparison | % Diff | -3.97 | -1.95 | 24.77 |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c | 3 | Latency 50th  | 393.55 | 0.31 | 1.24 | 1.53 | 1.32 | 0.31 | 
| 28c_con1 | 2 | SafeAverage of MedianThroughputCostPercent  | 30.39 | 1.98 | 2.58 | 2.41 | 1.98 | 2.48 | 
| 28c_con1 | 2 | Latency 50th  | 23.02 | 1.55 | 1.37 | 1.31 | 1.55 | 1.26 | 
| 28c | 1 | SafeAverage of MedianThroughputCostPercent  | 5.67 | 2.37 | 2.50 | 2.37 | 2.47 | 2.49 | 
### Bimodal Behavior

| Config | Clusters | Metric | Vol % | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ------ | ------ | ------ | ------ |
| 28c | (0.7, 0.71),(0.74, 0.72) | g0 Mean Pause  | 6.73 | 0.70 | 0.74 | 0.72 | 0.71 | 
| 28c_con1 | (0.7, 0.7),(0.72, 0.71) | g0 Mean Pause  | 3.52 | 0.70 | 0.70 | 0.72 | 0.71 | 
## JsonHttpsHttpSys

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | 7.88 | 1.44 | 0.58 |
| 28c | 1 | 7.97 | 1.47 | 0.58 |
| 28c | 2 | 7.95 | 1.48 | 0.58 |
| 28c | 3 | 8.16 | 1.46 | 0.58 |
| 28c_con1 | 0 | 5.95 | 1.50 | 0.56 |
| 28c_con1 | 1 | 5.83 | 1.48 | 0.56 |
| 28c_con1 | 2 | 5.99 | 1.52 | 0.56 |
| 28c_con1 | 3 | 5.99 | 1.48 | 0.55 |
| 28c | Average | 7.99 | 1.46 | 0.58 |
| 28c_con1 | Average | 5.94 | 1.50 | 0.56 |
| Comparison | % Diff | -25.62 | 2.40 | -3.88 |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c | 3 | SafeAverage of MedianThroughputCostPercent  | 3.53 | 8.16 | 7.88 | 7.97 | 7.95 | 8.16 | 
| 28c_con1 | 1 | SafeAverage of MedianThroughputCostPercent  | 2.78 | 5.83 | 5.95 | 5.83 | 5.99 | 5.99 | 
| 28c | 0 | g0 Mean Pause  | 2.76 | 1.44 | 1.44 | 1.47 | 1.48 | 1.46 | 
### Bimodal Behavior

| Config | Clusters | Metric | Vol % | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ------ | ------ | ------ | ------ |
| 28c_con1 | (1.5, 1.52),(1.48, 1.48) | g0 Mean Pause  | 2.39 | 1.50 | 1.48 | 1.52 | 1.48 | 
## JsonHttpSys

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | 7.57 | 1.46 | 0.43 |
| 28c | 1 | 7.17 | 1.42 | 0.50 |
| 28c | 2 | 6.91 | 1.38 | 0.43 |
| 28c | 3 | 5.39 | 1.26 | 0.75 |
| 28c_con1 | 0 | 5.28 | 1.45 | 0.47 |
| 28c_con1 | 1 | 5.02 | 1.47 | 0.55 |
| 28c_con1 | 2 | 4.15 | 1.37 | 0.70 |
| 28c_con1 | 3 | 3.55 | 1.32 | 0.76 |
| 28c | Average | 6.76 | 1.38 | 0.53 |
| 28c_con1 | Average | 4.50 | 1.40 | 0.62 |
| Comparison | % Diff | -33.47 | 1.55 | 17.54 |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c | 3 | Latency 50th  | 74.42 | 0.75 | 0.43 | 0.50 | 0.43 | 0.75 | 
| 28c | 3 | SafeAverage of MedianThroughputCostPercent  | 40.58 | 5.39 | 7.57 | 7.17 | 6.91 | 5.39 | 
| 28c | 3 | g0 Mean Pause  | 15.81 | 1.26 | 1.46 | 1.42 | 1.38 | 1.26 | 
### Bimodal Behavior

| Config | Clusters | Metric | Vol % | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ------ | ------ | ------ | ------ |
| 28c_con1 | (0.7, 0.76),(0.47, 0.55) | Latency 50th  | 61.7 | 0.47 | 0.55 | 0.70 | 0.76 | 
| 28c_con1 | (5.28, 5.02),(4.15, 3.55) | SafeAverage of MedianThroughputCostPercent  | 48.87 | 5.28 | 5.02 | 4.15 | 3.55 | 
| 28c_con1 | (1.37, 1.32),(1.45, 1.47) | g0 Mean Pause  | 11.05 | 1.45 | 1.47 | 1.37 | 1.32 | 
## JsonHttps

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | NaN | 8.00 | 0.31 |
| 28c | 1 | NaN | 7.81 | 0.32 |
| 28c | 2 | NaN | 5.88 | 0.32 |
| 28c | 3 | NaN | 5.91 | 0.31 |
| 28c_con1 | 0 | NaN | 8.30 | 0.32 |
| 28c_con1 | 1 | NaN | 5.81 | 0.32 |
| 28c_con1 | 2 | NaN | 5.97 | 0.31 |
| 28c_con1 | 3 | NaN | 8.18 | 0.30 |
| 28c | Average | NaN | 6.90 | 0.32 |
| 28c_con1 | Average | NaN | 7.06 | 0.31 |
| Comparison | % Diff | NaN | 2.37 | -0.79 |


### Bimodal Behavior

| Config | Clusters | Metric | Vol % | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ------ | ------ | ------ | ------ |
| 28c_con1 | (5.81, 5.97),(8.3, 8.18) | g0 Mean Pause  | 42.79 | 8.30 | 5.81 | 5.97 | 8.18 | 
| 28c | (5.88, 5.91),(8, 7.81) | g0 Mean Pause  | 36.21 | 8.00 | 7.81 | 5.88 | 5.91 | 
| 28c | (0.31, 0.31),(0.32, 0.32) | Latency 50th  | 3.23 | 0.31 | 0.32 | 0.32 | 0.31 | 
## JsonPlatformInline

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | NaN | 5.33 | 0.65 |
| 28c | 1 | NaN | 5.26 | 0.66 |
| 28c | 2 | NaN | 5.67 | 0.66 |
| 28c | 3 | NaN | 5.37 | 0.67 |
| 28c_con1 | 0 | NaN | 5.25 | 0.65 |
| 28c_con1 | 1 | NaN | 6.15 | 0.66 |
| 28c_con1 | 2 | NaN | 5.58 | 0.64 |
| 28c_con1 | 3 | NaN | 6.21 | 0.67 |
| 28c | Average | NaN | 5.41 | 0.66 |
| 28c_con1 | Average | NaN | 5.80 | 0.66 |
| Comparison | % Diff | NaN | 7.22 | -0.76 |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c | 2 | g0 Mean Pause  | 7.83 | 5.67 | 5.33 | 5.26 | 5.67 | 5.37 | 
| 28c_con1 | 3 | Latency 50th  | 4.69 | 0.67 | 0.65 | 0.66 | 0.64 | 0.67 | 
| 28c | 3 | Latency 50th  | 3.08 | 0.67 | 0.65 | 0.66 | 0.66 | 0.67 | 
### Bimodal Behavior

| Config | Clusters | Metric | Vol % | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ------ | ------ | ------ | ------ |
| 28c_con1 | (5.25, 5.58),(6.15, 6.21) | g0 Mean Pause  | 18.46 | 5.25 | 6.15 | 5.58 | 6.21 | 
## JsonPlatform

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | NaN | 6.10 | 0.32 |
| 28c | 1 | NaN | 5.94 | 0.29 |
| 28c | 2 | NaN | 8.23 | 0.29 |
| 28c | 3 | NaN | 6.29 | 0.29 |
| 28c_con1 | 0 | NaN | 6.39 | 0.31 |
| 28c_con1 | 1 | NaN | 6.15 | 0.28 |
| 28c_con1 | 2 | NaN | 6.43 | 0.29 |
| 28c_con1 | 3 | NaN | 4.85 | 0.32 |
| 28c | Average | NaN | 6.64 | 0.30 |
| 28c_con1 | Average | NaN | 5.96 | 0.30 |
| Comparison | % Diff | NaN | -10.27 | 0.84 |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c | 2 | g0 Mean Pause  | 38.57 | 8.23 | 6.10 | 5.94 | 8.23 | 6.29 | 
| 28c_con1 | 3 | g0 Mean Pause  | 32.63 | 4.85 | 6.39 | 6.15 | 6.43 | 4.85 | 
### Bimodal Behavior

| Config | Clusters | Metric | Vol % | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ------ | ------ | ------ | ------ |
| 28c_con1 | (0.31, 0.32),(0.28, 0.29) | Latency 50th  | 14.29 | 0.31 | 0.28 | 0.29 | 0.32 | 
## Json

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | NaN | 6.61 | 0.20 |
| 28c | 1 | NaN | 6.42 | 0.27 |
| 28c | 2 | NaN | 6.69 | 0.20 |
| 28c | 3 | NaN | 6.26 | 0.24 |
| 28c_con1 | 0 | NaN | 6.65 | 0.27 |
| 28c_con1 | 1 | NaN | 6.41 | 0.26 |
| 28c_con1 | 2 | NaN | 6.44 | 0.25 |
| 28c_con1 | 3 | NaN | 6.40 | 0.24 |
| 28c | Average | NaN | 6.50 | 0.23 |
| 28c_con1 | Average | NaN | 6.47 | 0.26 |
| Comparison | % Diff | NaN | -0.33 | 12.09 |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c_con1 | 0 | Latency 50th  | 12.5 | 0.27 | 0.27 | 0.26 | 0.25 | 0.24 | 
| 28c_con1 | 0 | g0 Mean Pause  | 3.89 | 6.65 | 6.65 | 6.41 | 6.44 | 6.40 | 
### Bimodal Behavior

| Config | Clusters | Metric | Vol % | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ------ | ------ | ------ | ------ |
| 28c | (0.27, 0.24),(0.2, 0.2) | Latency 50th  | 35 | 0.20 | 0.27 | 0.20 | 0.24 | 
| 28c | (6.61, 6.69),(6.42, 6.26) | g0 Mean Pause  | 6.79 | 6.61 | 6.42 | 6.69 | 6.26 | 
## PlaintextEndpoint

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | NaN | 6.94 | 0.46 |
| 28c | 1 | NaN | 6.63 | 0.44 |
| 28c | 2 | NaN | 6.71 | 0.46 |
| 28c | 3 | NaN | 7.43 | 0.47 |
| 28c_con1 | 0 | NaN | 6.72 | 0.44 |
| 28c_con1 | 1 | NaN | 7.18 | 0.46 |
| 28c_con1 | 2 | NaN | 6.96 | 0.48 |
| 28c_con1 | 3 | NaN | 7.01 | 0.46 |
| 28c | Average | NaN | 6.93 | 0.46 |
| 28c_con1 | Average | NaN | 6.97 | 0.46 |
| Comparison | % Diff | NaN | 0.52 | 0.55 |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c | 3 | g0 Mean Pause  | 12.06 | 7.43 | 6.94 | 6.63 | 6.71 | 7.43 | 
| 28c_con1 | 0 | Latency 50th  | 9.09 | 0.44 | 0.44 | 0.46 | 0.48 | 0.46 | 
| 28c_con1 | 0 | g0 Mean Pause  | 6.83 | 6.72 | 6.72 | 7.18 | 6.96 | 7.01 | 
| 28c | 1 | Latency 50th  | 6.82 | 0.44 | 0.46 | 0.44 | 0.46 | 0.47 | 
## PlaintextMapAction

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | NaN | 5.25 | 0.54 |
| 28c | 1 | NaN | 5.17 | 0.55 |
| 28c | 2 | NaN | 5.31 | 0.55 |
| 28c | 3 | NaN | 5.36 | 0.55 |
| 28c_con1 | 0 | NaN | 5.41 | 0.54 |
| 28c_con1 | 1 | NaN | 5.29 | 0.56 |
| 28c_con1 | 2 | NaN | 5.30 | 0.55 |
| 28c_con1 | 3 | NaN | 5.50 | 0.54 |
| 28c | Average | NaN | 5.27 | 0.55 |
| 28c_con1 | Average | NaN | 5.38 | 0.55 |
| Comparison | % Diff | NaN | 2.02 | 0.00 |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c_con1 | 1 | Latency 50th  | 3.7 | 0.56 | 0.54 | 0.56 | 0.55 | 0.54 | 
### Bimodal Behavior

| Config | Clusters | Metric | Vol % | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ------ | ------ | ------ | ------ |
| 28c_con1 | (5.29, 5.3),(5.41, 5.5) | g0 Mean Pause  | 3.95 | 5.41 | 5.29 | 5.30 | 5.50 | 
| 28c | (5.31, 5.36),(5.25, 5.17) | g0 Mean Pause  | 3.68 | 5.25 | 5.17 | 5.31 | 5.36 | 
## PlaintextMvc

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | 13.14 | 1.37 | 0.82 |
| 28c | 1 | 13.73 | 1.42 | 0.81 |
| 28c | 2 | 12.27 | 1.32 | 0.81 |
| 28c | 3 | 14.66 | 1.58 | 0.83 |
| 28c_con1 | 0 | 9.39 | 1.57 | 0.78 |
| 28c_con1 | 1 | 9.99 | 1.61 | 0.78 |
| 28c_con1 | 2 | 8.66 | 1.47 | 0.77 |
| 28c_con1 | 3 | 9.50 | 1.55 | 0.77 |
| 28c | Average | 13.45 | 1.42 | 0.82 |
| 28c_con1 | Average | 9.39 | 1.55 | 0.78 |
| Comparison | % Diff | -30.22 | 9.25 | -5.20 |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c | 2 | SafeAverage of MedianThroughputCostPercent  | 19.5 | 12.27 | 13.14 | 13.73 | 12.27 | 14.66 | 
| 28c_con1 | 1 | SafeAverage of MedianThroughputCostPercent  | 15.35 | 9.99 | 9.39 | 9.99 | 8.66 | 9.50 | 
| 28c_con1 | 2 | g0 Mean Pause  | 10.01 | 1.47 | 1.57 | 1.61 | 1.47 | 1.55 | 
| 28c | 3 | Latency 50th  | 2.47 | 0.83 | 0.82 | 0.81 | 0.81 | 0.83 | 
### Bimodal Behavior

| Config | Clusters | Metric | Vol % | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ------ | ------ | ------ | ------ |
| 28c | (1.37, 1.32),(1.42, 1.58) | g0 Mean Pause  | 19.84 | 1.37 | 1.42 | 1.32 | 1.58 | 
| 28c_con1 | (0.78, 0.78),(0.77, 0.77) | Latency 50th  | 1.3 | 0.78 | 0.78 | 0.77 | 0.77 | 
## PlaintextNoParametersEmptyFilter

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | 2.08 | 1.96 | 0.59 |
| 28c | 1 | 1.89 | 1.52 | 0.60 |
| 28c | 2 | 1.88 | 1.60 | 0.58 |
| 28c | 3 | 1.92 | 1.61 | 0.60 |
| 28c_con1 | 0 | 2.02 | 1.93 | 0.58 |
| 28c_con1 | 1 | 1.86 | 1.57 | 0.59 |
| 28c_con1 | 2 | 1.92 | 1.68 | 0.60 |
| 28c_con1 | 3 | 1.86 | 1.65 | 0.60 |
| 28c | Average | 1.94 | 1.67 | 0.59 |
| 28c_con1 | Average | 1.92 | 1.71 | 0.59 |
| Comparison | % Diff | -1.45 | 2.02 | 0.00 |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c | 0 | g0 Mean Pause  | 29.55 | 1.96 | 1.96 | 1.52 | 1.60 | 1.61 | 
| 28c_con1 | 0 | g0 Mean Pause  | 22.79 | 1.93 | 1.93 | 1.57 | 1.68 | 1.65 | 
| 28c | 0 | SafeAverage of MedianThroughputCostPercent  | 10.74 | 2.08 | 2.08 | 1.89 | 1.88 | 1.92 | 
| 28c_con1 | 0 | SafeAverage of MedianThroughputCostPercent  | 8.98 | 2.02 | 2.02 | 1.86 | 1.92 | 1.86 | 
### Bimodal Behavior

| Config | Clusters | Metric | Vol % | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ------ | ------ | ------ | ------ |
| 28c | (0.59, 0.58),(0.6, 0.6) | Latency 50th  | 3.45 | 0.59 | 0.60 | 0.58 | 0.60 | 
| 28c_con1 | (0.58, 0.59),(0.6, 0.6) | Latency 50th  | 3.45 | 0.58 | 0.59 | 0.60 | 0.60 | 
## PlaintextPlatformInline

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | NaN | 6.35 | 1.02 |
| 28c | 1 | NaN | 8.42 | 1.04 |
| 28c | 2 | NaN | 6.13 | 1.04 |
| 28c | 3 | NaN | 6.20 | 1.05 |
| 28c_con1 | 0 | NaN | 5.57 | 1.03 |
| 28c_con1 | 1 | NaN | 19.52 | 1.04 |
| 28c_con1 | 2 | NaN | 6.08 | 1.05 |
| 28c_con1 | 3 | NaN | 5.47 | 1.06 |
| 28c | Average | NaN | 6.77 | 1.04 |
| 28c_con1 | Average | NaN | 9.16 | 1.04 |
| Comparison | % Diff | NaN | 35.24 | 0.72 |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c_con1 | 1 | g0 Mean Pause  | 256.73 | 19.52 | 5.57 | 19.52 | 6.08 | 5.47 | 
| 28c | 1 | g0 Mean Pause  | 37.36 | 8.42 | 6.35 | 8.42 | 6.13 | 6.20 | 
| 28c | 0 | Latency 50th  | 2.94 | 1.02 | 1.02 | 1.04 | 1.04 | 1.05 | 
### Bimodal Behavior

| Config | Clusters | Metric | Vol % | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ------ | ------ | ------ | ------ |
| 28c_con1 | (1.05, 1.06),(1.03, 1.04) | Latency 50th  | 2.91 | 1.03 | 1.04 | 1.05 | 1.06 | 
## PlaintextPlatform

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | NaN | 6.31 | 0.85 |
| 28c | 1 | NaN | 6.19 | 0.85 |
| 28c | 2 | NaN | 6.87 | 0.89 |
| 28c | 3 | NaN | 5.83 | 0.86 |
| 28c_con1 | 0 | NaN | 6.26 | 0.86 |
| 28c_con1 | 1 | NaN | 6.10 | 0.84 |
| 28c_con1 | 2 | NaN | 5.25 | 0.86 |
| 28c_con1 | 3 | NaN | 6.19 | 0.85 |
| 28c | Average | NaN | 6.30 | 0.86 |
| 28c_con1 | Average | NaN | 5.95 | 0.85 |
| Comparison | % Diff | NaN | -5.57 | -1.16 |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c_con1 | 2 | g0 Mean Pause  | 19.26 | 5.25 | 6.26 | 6.10 | 5.25 | 6.19 | 
| 28c_con1 | 1 | Latency 50th  | 2.38 | 0.84 | 0.86 | 0.84 | 0.86 | 0.85 | 
### Bimodal Behavior

| Config | Clusters | Metric | Vol % | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ------ | ------ | ------ | ------ |
| 28c | (6.19, 5.83),(6.31, 6.87) | g0 Mean Pause  | 17.8 | 6.31 | 6.19 | 6.87 | 5.83 | 
## PlaintextQueryString

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | 3.63 | 1.09 | 0.62 |
| 28c | 1 | 3.79 | 1.09 | 0.60 |
| 28c | 2 | 7.20 | 1.30 | 0.61 |
| 28c | 3 | 7.40 | 1.32 | 0.61 |
| 28c_con1 | 0 | 2.97 | 1.23 | 0.59 |
| 28c_con1 | 1 | 3.02 | 1.24 | 0.60 |
| 28c_con1 | 2 | 3.31 | 1.33 | 0.60 |
| 28c_con1 | 3 | 3.21 | 1.31 | 0.59 |
| 28c | Average | 5.51 | 1.20 | 0.61 |
| 28c_con1 | Average | 3.13 | 1.28 | 0.59 |
| Comparison | % Diff | -43.25 | 6.84 | -2.46 |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c | 1 | Latency 50th  | 3.33 | 0.6 | 0.62 | 0.60 | 0.61 | 0.61 | 
### Bimodal Behavior

| Config | Clusters | Metric | Vol % | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ------ | ------ | ------ | ------ |
| 28c | (7.2, 7.4),(3.63, 3.79) | SafeAverage of MedianThroughputCostPercent  | 103.85 | 3.63 | 3.79 | 7.20 | 7.40 | 
| 28c | (1.3, 1.32),(1.09, 1.09) | g0 Mean Pause  | 21.08 | 1.09 | 1.09 | 1.30 | 1.32 | 
| 28c_con1 | (2.97, 3.02),(3.31, 3.21) | SafeAverage of MedianThroughputCostPercent  | 11.71 | 2.97 | 3.02 | 3.31 | 3.21 | 
| 28c_con1 | (1.33, 1.31),(1.23, 1.24) | g0 Mean Pause  | 8.07 | 1.23 | 1.24 | 1.33 | 1.31 | 
| 28c_con1 | (0.59, 0.59),(0.6, 0.6) | Latency 50th  | 1.69 | 0.59 | 0.60 | 0.60 | 0.59 | 
## PlaintextWithParametersEmptyFilter

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | 5.38 | 1.55 | 0.68 |
| 28c | 1 | 4.34 | 1.35 | 0.72 |
| 28c | 2 | 4.38 | 1.35 | 0.67 |
| 28c | 3 | 4.13 | 1.31 | 0.70 |
| 28c_con1 | 0 | 3.00 | 1.37 | 0.67 |
| 28c_con1 | 1 | 3.23 | 1.41 | 0.66 |
| 28c_con1 | 2 | 3.79 | 1.58 | 0.68 |
| 28c_con1 | 3 | 3.27 | 1.45 | 0.67 |
| 28c | Average | 4.56 | 1.39 | 0.69 |
| 28c_con1 | Average | 3.32 | 1.46 | 0.67 |
| Comparison | % Diff | -27.08 | 4.57 | -3.25 |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c | 0 | SafeAverage of MedianThroughputCostPercent  | 30.51 | 5.38 | 5.38 | 4.34 | 4.38 | 4.13 | 
| 28c_con1 | 2 | SafeAverage of MedianThroughputCostPercent  | 26.29 | 3.79 | 3.00 | 3.23 | 3.79 | 3.27 | 
| 28c | 0 | g0 Mean Pause  | 18.32 | 1.55 | 1.55 | 1.35 | 1.35 | 1.31 | 
| 28c_con1 | 2 | g0 Mean Pause  | 15.73 | 1.58 | 1.37 | 1.41 | 1.58 | 1.45 | 
| 28c_con1 | 2 | Latency 50th  | 3.03 | 0.68 | 0.67 | 0.66 | 0.68 | 0.67 | 
### Bimodal Behavior

| Config | Clusters | Metric | Vol % | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ------ | ------ | ------ | ------ |
| 28c | (0.68, 0.67),(0.72, 0.7) | Latency 50th  | 7.46 | 0.68 | 0.72 | 0.67 | 0.70 | 
## PlaintextWithParametersNoFilter

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | 5.14 | 1.55 | 0.65 |
| 28c | 1 | 4.98 | 1.54 | 0.65 |
| 28c | 2 | 4.27 | 1.37 | 0.65 |
| 28c | 3 | 4.11 | 1.36 | 0.64 |
| 28c_con1 | 0 | 3.62 | 1.61 | 0.67 |
| 28c_con1 | 1 | 3.50 | 1.58 | 0.66 |
| 28c_con1 | 2 | 3.64 | 1.62 | 0.63 |
| 28c_con1 | 3 | 3.60 | 1.62 | 0.64 |
| 28c | Average | 4.63 | 1.46 | 0.65 |
| 28c_con1 | Average | 3.59 | 1.61 | 0.65 |
| Comparison | % Diff | -22.44 | 10.40 | 0.39 |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c_con1 | 1 | SafeAverage of MedianThroughputCostPercent  | 4.08 | 3.5 | 3.62 | 3.50 | 3.64 | 3.60 | 
| 28c_con1 | 1 | g0 Mean Pause  | 2.33 | 1.58 | 1.61 | 1.58 | 1.62 | 1.62 | 
| 28c | 3 | Latency 50th  | 1.56 | 0.64 | 0.65 | 0.65 | 0.65 | 0.64 | 
### Bimodal Behavior

| Config | Clusters | Metric | Vol % | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ------ | ------ | ------ | ------ |
| 28c | (4.27, 4.11),(5.14, 4.98) | SafeAverage of MedianThroughputCostPercent  | 24.96 | 5.14 | 4.98 | 4.27 | 4.11 | 
| 28c | (1.55, 1.54),(1.37, 1.36) | g0 Mean Pause  | 14.26 | 1.55 | 1.54 | 1.37 | 1.36 | 
| 28c_con1 | (0.67, 0.66),(0.63, 0.64) | Latency 50th  | 6.35 | 0.67 | 0.66 | 0.63 | 0.64 | 
## PlaintextWithParameterstWithFilter

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | 5.39 | 1.39 | 0.71 |
| 28c | 1 | 7.26 | 1.75 | 0.76 |
| 28c | 2 | 4.86 | 1.35 | 0.73 |
| 28c | 3 | 5.65 | 1.44 | 0.71 |
| 28c_con1 | 0 | 4.07 | 1.59 | 0.70 |
| 28c_con1 | 1 | 4.08 | 1.60 | 0.69 |
| 28c_con1 | 2 | 3.08 | 1.30 | 0.70 |
| 28c_con1 | 3 | 2.37 | 1.29 | 0.68 |
| 28c | Average | 5.79 | 1.48 | 0.73 |
| 28c_con1 | Average | 3.40 | 1.45 | 0.69 |
| Comparison | % Diff | -41.30 | -2.47 | -4.81 |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c | 1 | SafeAverage of MedianThroughputCostPercent  | 49.54 | 7.26 | 5.39 | 7.26 | 4.86 | 5.65 | 
| 28c | 1 | g0 Mean Pause  | 30.4 | 1.75 | 1.39 | 1.75 | 1.35 | 1.44 | 
| 28c | 1 | Latency 50th  | 7.04 | 0.76 | 0.71 | 0.76 | 0.73 | 0.71 | 
| 28c_con1 | 3 | Latency 50th  | 2.94 | 0.68 | 0.70 | 0.69 | 0.70 | 0.68 | 
### Bimodal Behavior

| Config | Clusters | Metric | Vol % | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ------ | ------ | ------ | ------ |
| 28c_con1 | (4.07, 4.08),(3.08, 2.37) | SafeAverage of MedianThroughputCostPercent  | 71.87 | 4.07 | 4.08 | 3.08 | 2.37 | 
| 28c_con1 | (1.59, 1.6),(1.3, 1.29) | g0 Mean Pause  | 23.61 | 1.59 | 1.60 | 1.30 | 1.29 | 
## Plaintext

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | NaN | 6.61 | 0.42 |
| 28c | 1 | NaN | 7.14 | 0.41 |
| 28c | 2 | NaN | 6.50 | 0.41 |
| 28c | 3 | NaN | 6.88 | 0.41 |
| 28c_con1 | 0 | NaN | 6.95 | 0.41 |
| 28c_con1 | 1 | NaN | 6.72 | 0.41 |
| 28c_con1 | 2 | NaN | 6.75 | 0.40 |
| 28c_con1 | 3 | NaN | 6.10 | 0.43 |
| 28c | Average | NaN | 6.78 | 0.41 |
| 28c_con1 | Average | NaN | 6.63 | 0.41 |
| Comparison | % Diff | NaN | -2.21 | 0.00 |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c_con1 | 3 | g0 Mean Pause  | 13.89 | 6.1 | 6.95 | 6.72 | 6.75 | 6.10 | 
| 28c_con1 | 3 | Latency 50th  | 7.5 | 0.43 | 0.41 | 0.41 | 0.40 | 0.43 | 
### Bimodal Behavior

| Config | Clusters | Metric | Vol % | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ------ | ------ | ------ | ------ |
| 28c | (7.14, 6.88),(6.61, 6.5) | g0 Mean Pause  | 9.75 | 6.61 | 7.14 | 6.50 | 6.88 | 
## Stage1GrpcPgo

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | 4.66 | 3.00 | NaN |
| 28c | 1 | 4.16 | 2.90 | NaN |
| 28c | 2 | 4.55 | 2.98 | NaN |
| 28c | 3 | 4.52 | 2.78 | NaN |
| 28c_con1 | 0 | 4.70 | 3.05 | NaN |
| 28c_con1 | 1 | 3.91 | 2.95 | NaN |
| 28c_con1 | 2 | 4.46 | 2.98 | NaN |
| 28c_con1 | 3 | 4.28 | 2.98 | NaN |
| 28c | Average | 4.47 | 2.92 | NaN |
| 28c_con1 | Average | 4.34 | 2.99 | NaN |
| Comparison | % Diff | -3.02 | 2.60 | NaN |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c | 1 | SafeAverage of MedianThroughputCostPercent  | 12.02 | 4.16 | 4.66 | 4.16 | 4.55 | 4.52 | 
| 28c_con1 | 0 | g0 Mean Pause  | 3.33 | 3.05 | 3.05 | 2.95 | 2.98 | 2.98 | 
### Bimodal Behavior

| Config | Clusters | Metric | Vol % | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ------ | ------ | ------ | ------ |
| 28c_con1 | (4.7, 4.46),(3.91, 4.28) | SafeAverage of MedianThroughputCostPercent  | 20.25 | 4.70 | 3.91 | 4.46 | 4.28 | 
| 28c | (3, 2.98),(2.9, 2.78) | g0 Mean Pause  | 8.07 | 3.00 | 2.90 | 2.98 | 2.78 | 
## Stage1GrpcTrimR2RSingleFile

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | 4.52 | 2.83 | NaN |
| 28c | 1 | 4.49 | 2.95 | NaN |
| 28c | 2 | 4.29 | 3.05 | NaN |
| 28c | 3 | 4.24 | 2.80 | NaN |
| 28c_con1 | 0 | 4.17 | 2.99 | NaN |
| 28c_con1 | 1 | 3.55 | 3.10 | NaN |
| 28c_con1 | 2 | 3.56 | 3.03 | NaN |
| 28c_con1 | 3 | 3.41 | 3.09 | NaN |
| 28c | Average | 4.38 | 2.91 | NaN |
| 28c_con1 | Average | 3.67 | 3.05 | NaN |
| Comparison | % Diff | -16.20 | 4.97 | NaN |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c_con1 | 0 | SafeAverage of MedianThroughputCostPercent  | 22.4 | 4.17 | 4.17 | 3.55 | 3.56 | 3.41 | 
### Bimodal Behavior

| Config | Clusters | Metric | Vol % | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ------ | ------ | ------ | ------ |
| 28c | (2.95, 3.05),(2.83, 2.8) | g0 Mean Pause  | 8.98 | 2.83 | 2.95 | 3.05 | 2.80 | 
| 28c | (4.52, 4.49),(4.29, 4.24) | SafeAverage of MedianThroughputCostPercent  | 6.61 | 4.52 | 4.49 | 4.29 | 4.24 | 
| 28c_con1 | (3.1, 3.09),(2.99, 3.03) | g0 Mean Pause  | 3.71 | 2.99 | 3.10 | 3.03 | 3.09 | 
## Stage1Grpc

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | 4.55 | 2.95 | NaN |
| 28c | 1 | 4.41 | 2.80 | NaN |
| 28c | 2 | 4.48 | 2.90 | NaN |
| 28c | 3 | 4.75 | 2.98 | NaN |
| 28c_con1 | 0 | 4.08 | 2.87 | NaN |
| 28c_con1 | 1 | 4.61 | 2.98 | NaN |
| 28c_con1 | 2 | 4.62 | 3.00 | NaN |
| 28c_con1 | 3 | 4.04 | 2.97 | NaN |
| 28c | Average | 4.55 | 2.91 | NaN |
| 28c_con1 | Average | 4.34 | 2.96 | NaN |
| Comparison | % Diff | -4.62 | 1.71 | NaN |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c | 3 | SafeAverage of MedianThroughputCostPercent  | 7.56 | 4.75 | 4.55 | 4.41 | 4.48 | 4.75 | 
| 28c | 1 | g0 Mean Pause  | 6.36 | 2.8 | 2.95 | 2.80 | 2.90 | 2.98 | 
| 28c_con1 | 0 | g0 Mean Pause  | 4.67 | 2.87 | 2.87 | 2.98 | 3.00 | 2.97 | 
### Bimodal Behavior

| Config | Clusters | Metric | Vol % | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ------ | ------ | ------ | ------ |
| 28c_con1 | (4.08, 4.04),(4.61, 4.62) | SafeAverage of MedianThroughputCostPercent  | 14.48 | 4.08 | 4.61 | 4.62 | 4.04 | 
## Stage1Pgo

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | 1.96 | 1.35 | 0.30 |
| 28c | 1 | 1.93 | 1.50 | 0.28 |
| 28c | 2 | 1.94 | 1.38 | 0.27 |
| 28c | 3 | 1.97 | 1.38 | 0.28 |
| 28c_con1 | 0 | 1.40 | 1.24 | 0.32 |
| 28c_con1 | 1 | 2.04 | 1.37 | 0.28 |
| 28c_con1 | 2 | 1.86 | 1.26 | 0.32 |
| 28c_con1 | 3 | 1.83 | 1.49 | 0.27 |
| 28c | Average | 1.95 | 1.40 | 0.28 |
| 28c_con1 | Average | 1.78 | 1.34 | 0.30 |
| Comparison | % Diff | -8.70 | -4.48 | 5.31 |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c_con1 | 0 | SafeAverage of MedianThroughputCostPercent  | 45.68 | 1.4 | 1.40 | 2.04 | 1.86 | 1.83 | 
| 28c | 2 | Latency 50th  | 11.11 | 0.27 | 0.30 | 0.28 | 0.27 | 0.28 | 
| 28c | 1 | g0 Mean Pause  | 11.07 | 1.5 | 1.35 | 1.50 | 1.38 | 1.38 | 
### Bimodal Behavior

| Config | Clusters | Metric | Vol % | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ------ | ------ | ------ | ------ |
| 28c_con1 | (1.24, 1.26),(1.37, 1.49) | g0 Mean Pause  | 20.74 | 1.24 | 1.37 | 1.26 | 1.49 | 
| 28c_con1 | (0.28, 0.27),(0.32, 0.32) | Latency 50th  | 18.52 | 0.32 | 0.28 | 0.32 | 0.27 | 
| 28c | (1.96, 1.97),(1.93, 1.94) | SafeAverage of MedianThroughputCostPercent  | 2.07 | 1.96 | 1.93 | 1.94 | 1.97 | 
## Stage1TrimR2RSingleFile

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | 1.92 | 1.45 | 0.29 |
| 28c | 1 | 2.16 | 1.15 | 0.28 |
| 28c | 2 | 1.95 | 1.37 | 0.26 |
| 28c | 3 | 2.07 | 1.24 | 0.30 |
| 28c_con1 | 0 | 1.73 | 1.34 | 0.29 |
| 28c_con1 | 1 | 1.87 | 1.37 | 0.27 |
| 28c_con1 | 2 | 2.29 | 1.47 | 0.31 |
| 28c_con1 | 3 | 1.83 | 1.28 | 0.26 |
| 28c | Average | 2.03 | 1.30 | 0.28 |
| 28c_con1 | Average | 1.93 | 1.37 | 0.28 |
| Comparison | % Diff | -4.73 | 5.02 | 0.00 |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c_con1 | 2 | SafeAverage of MedianThroughputCostPercent  | 32 | 2.29 | 1.73 | 1.87 | 2.29 | 1.83 | 
| 28c | 2 | Latency 50th  | 15.38 | 0.26 | 0.29 | 0.28 | 0.26 | 0.30 | 
| 28c_con1 | 3 | g0 Mean Pause  | 14.64 | 1.28 | 1.34 | 1.37 | 1.47 | 1.28 | 
### Bimodal Behavior

| Config | Clusters | Metric | Vol % | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ------ | ------ | ------ | ------ |
| 28c | (1.45, 1.37),(1.15, 1.24) | g0 Mean Pause  | 25.78 | 1.45 | 1.15 | 1.37 | 1.24 | 
| 28c_con1 | (0.29, 0.31),(0.27, 0.26) | Latency 50th  | 19.23 | 0.29 | 0.27 | 0.31 | 0.26 | 
| 28c | (1.92, 1.95),(2.16, 2.07) | SafeAverage of MedianThroughputCostPercent  | 12.73 | 1.92 | 2.16 | 1.95 | 2.07 | 
## Stage1

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | 1.87 | 1.41 | 0.29 |
| 28c | 1 | 2.04 | 1.46 | 0.27 |
| 28c | 2 | 2.05 | 1.37 | 0.27 |
| 28c | 3 | 1.71 | 1.31 | 0.30 |
| 28c_con1 | 0 | 2.27 | 1.78 | 0.28 |
| 28c_con1 | 1 | 1.77 | 1.42 | 0.25 |
| 28c_con1 | 2 | 1.84 | 1.47 | 0.29 |
| 28c_con1 | 3 | 2.03 | 1.59 | 0.28 |
| 28c | Average | 1.92 | 1.39 | 0.28 |
| 28c_con1 | Average | 1.98 | 1.56 | 0.28 |
| Comparison | % Diff | 3.09 | 12.56 | -2.65 |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c_con1 | 0 | SafeAverage of MedianThroughputCostPercent  | 28.32 | 2.27 | 2.27 | 1.77 | 1.84 | 2.03 | 
| 28c | 3 | SafeAverage of MedianThroughputCostPercent  | 20.28 | 1.71 | 1.87 | 2.04 | 2.05 | 1.71 | 
| 28c_con1 | 1 | Latency 50th  | 16 | 0.25 | 0.28 | 0.25 | 0.29 | 0.28 | 
| 28c | 3 | g0 Mean Pause  | 11.63 | 1.31 | 1.41 | 1.46 | 1.37 | 1.31 | 
### Bimodal Behavior

| Config | Clusters | Metric | Vol % | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ------ | ------ | ------ | ------ |
| 28c_con1 | (1.42, 1.47),(1.78, 1.59) | g0 Mean Pause  | 25.43 | 1.78 | 1.42 | 1.47 | 1.59 | 
| 28c | (0.27, 0.27),(0.29, 0.3) | Latency 50th  | 11.11 | 0.29 | 0.27 | 0.27 | 0.30 | 
## Stage2Pgo

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | 3.58 | 1.94 | 1.12 |
| 28c | 1 | 4.02 | 1.94 | 1.12 |
| 28c | 2 | 3.92 | 2.00 | 1.11 |
| 28c | 3 | 4.05 | 1.92 | 1.12 |
| 28c_con1 | 0 | 3.10 | 2.06 | 1.12 |
| 28c_con1 | 1 | 2.94 | 2.02 | 1.11 |
| 28c_con1 | 2 | 2.98 | 2.06 | 1.12 |
| 28c_con1 | 3 | 3.00 | 2.14 | 1.11 |
| 28c | Average | 3.89 | 1.95 | 1.12 |
| 28c_con1 | Average | 3.00 | 2.07 | 1.12 |
| Comparison | % Diff | -22.82 | 6.09 | -0.22 |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c | 0 | SafeAverage of MedianThroughputCostPercent  | 13.03 | 3.58 | 3.58 | 4.02 | 3.92 | 4.05 | 
| 28c_con1 | 1 | g0 Mean Pause  | 5.81 | 2.02 | 2.06 | 2.02 | 2.06 | 2.14 | 
| 28c_con1 | 0 | SafeAverage of MedianThroughputCostPercent  | 5.13 | 3.1 | 3.10 | 2.94 | 2.98 | 3.00 | 
| 28c | 3 | g0 Mean Pause  | 4.09 | 1.92 | 1.94 | 1.94 | 2.00 | 1.92 | 
| 28c | 2 | Latency 50th  | 0.9 | 1.11 | 1.12 | 1.12 | 1.11 | 1.12 | 
### Bimodal Behavior

| Config | Clusters | Metric | Vol % | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ------ | ------ | ------ | ------ |
| 28c_con1 | (1.11, 1.11),(1.12, 1.12) | Latency 50th  | 0.9 | 1.12 | 1.11 | 1.12 | 1.11 | 
## Stage2TrimR2RSingleFile

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | 3.40 | 1.84 | 1.13 |
| 28c | 1 | 3.94 | 1.92 | 1.14 |
| 28c | 2 | 3.59 | 1.86 | 1.14 |
| 28c | 3 | 3.55 | 1.88 | 1.13 |
| 28c_con1 | 0 | 2.68 | 1.87 | 1.13 |
| 28c_con1 | 1 | 3.20 | 1.91 | 1.13 |
| 28c_con1 | 2 | 2.88 | 1.95 | 1.13 |
| 28c_con1 | 3 | 3.00 | 2.00 | 1.13 |
| 28c | Average | 3.62 | 1.88 | 1.13 |
| 28c_con1 | Average | 2.94 | 1.93 | 1.13 |
| Comparison | % Diff | -18.80 | 3.02 | -0.44 |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c | 1 | SafeAverage of MedianThroughputCostPercent  | 15.7 | 3.94 | 3.40 | 3.94 | 3.59 | 3.55 | 
### Bimodal Behavior

| Config | Clusters | Metric | Vol % | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ------ | ------ | ------ | ------ |
| 28c_con1 | (2.68, 2.88),(3.2, 3) | SafeAverage of MedianThroughputCostPercent  | 19.16 | 2.68 | 3.20 | 2.88 | 3.00 | 
| 28c_con1 | (1.87, 1.91),(1.95, 2) | g0 Mean Pause  | 6.95 | 1.87 | 1.91 | 1.95 | 2.00 | 
| 28c | (1.84, 1.86),(1.92, 1.88) | g0 Mean Pause  | 4.09 | 1.84 | 1.92 | 1.86 | 1.88 | 
## Stage2

| Config | Iteration | SafeAverage of MedianThroughputCostPercent | g0 Mean Pause | Latency 50th |
| ------ | --------- | ------ | ------ | ------ |
| 28c | 0 | 3.82 | 1.93 | 1.11 |
| 28c | 1 | 3.89 | 1.91 | 1.10 |
| 28c | 2 | 3.60 | 1.91 | 1.11 |
| 28c | 3 | 3.57 | 1.90 | 1.12 |
| 28c_con1 | 0 | 2.95 | 2.02 | 1.11 |
| 28c_con1 | 1 | 2.91 | 2.07 | 1.11 |
| 28c_con1 | 2 | 3.26 | 2.12 | 1.11 |
| 28c_con1 | 3 | 2.98 | 2.05 | 1.11 |
| 28c | Average | 3.72 | 1.91 | 1.11 |
| 28c_con1 | Average | 3.03 | 2.06 | 1.11 |
| Comparison | % Diff | -18.70 | 7.84 | 0.00 |


### Anomalies

| Config | Iteration | Metric | Vol % | Anomalous Value | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ----------- | ------ | ------ | ------ | ------ |
| 28c_con1 | 2 | SafeAverage of MedianThroughputCostPercent  | 11.87 | 3.26 | 2.95 | 2.91 | 3.26 | 2.98 | 
| 28c_con1 | 2 | g0 Mean Pause  | 4.98 | 2.12 | 2.02 | 2.07 | 2.12 | 2.05 | 
| 28c | 1 | Latency 50th  | 1.82 | 1.1 | 1.11 | 1.10 | 1.11 | 1.12 | 
| 28c | 0 | g0 Mean Pause  | 1.35 | 1.93 | 1.93 | 1.91 | 1.91 | 1.90 | 
### Bimodal Behavior

| Config | Clusters | Metric | Vol % | 0|1|2|3 |
| ------ | --------- | ------ | ---------- | ------ | ------ | ------ | ------ |
| 28c | (3.82, 3.89),(3.6, 3.57) | SafeAverage of MedianThroughputCostPercent  | 8.9 | 3.82 | 3.89 | 3.60 | 3.57 | 


## Display GC Charts

In [26]:
var GCPauseTimeMetricFilter = ML( Metrics.G.GCPauseTime );
ChartGCData(dm, metrics: GCPauseTimeMetricFilter, benchmarkFilter: benchmarkFilter, xMetric: Metrics.X.StartRelativeMSec)
.ForEach(ch => ch.Show());

var HeapSizeMetricFilter = ML( Metrics.G.HeapSizeBefore, Metrics.G.HeapSizeAfter, Metrics.G.HeapSizePeak );
ChartGCData(dm, metrics: HeapSizeMetricFilter, benchmarkFilter: benchmarkFilter, xMetric: Metrics.X.StartRelativeMSec)
.ForEach(ch => ch.Show());

Height,500
Id,c9be4a65-fd56-4fc7-b701-2930dff2c1f7
PlotlySrc,https://cdn.plot.ly/plotly-latest.min.js
Width,900


Height,500
Id,eeea6bf9-8a0f-4f9b-bab0-1d284eeef2e5
PlotlySrc,https://cdn.plot.ly/plotly-latest.min.js
Width,900
